Integrantes: 
- Oscar Estrada
- Sara Paguaga
- Guillermo Santos

Enlace github: https://github.com/moraleseoscar/CalCOFIWater_Salinity

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import os

In [4]:
bottle_df = pd.read_csv('project/CLEAN/bottle_clean.csv')
bottle_df.head(2)

,Salnty,T_degC,Depthm,O2ml_L,STheta,O2Sat
0,33.03,10.30,0,5.90,25.364,92.8
1,32.92,18.46,6,6.02,23.568,111.8


In [5]:
bottle_df.dropna(inplace=True)

In [6]:
X = bottle_df[['T_degC', 'Depthm', 'O2ml_L', 'STheta', 'O2Sat']]
y = bottle_df['Salnty']

In [7]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Pipeline

In [9]:
# Create a pipeline with scaling and the MLPRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42))
])

# Fit the model on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000,
                              random_state=42))])

In [10]:
# Predict on the test data
y_pred = pipeline.predict(X_test)

In [11]:
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

Mean Squared Error: 0.0005392154704440376
R-squared: 0.9974488199493196


## Calibrating the model and improving metrics

In [15]:
# Establishing a grid of hyperparameters to test
options_grid = {
    'hidden_layer_sizes': [(100, 50), (50, 50), (50, 100, 50)],
    'activation': ['tanh', 'relu'],
    'alpha': [0.0001, 0.05, 0.000001]
}

# Initialize the MLPRegressor
mlpr = MLPRegressor(max_iter=1000, random_state=42)

# Perform a grid search with the options in options_grid
grid_search = GridSearchCV(mlpr, options_grid, cv=5, n_jobs=-1)

# Fit the grid search
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'activation': 'tanh', 'alpha': 1e-06, 'hidden_layer_sizes': (50, 50)}


In [17]:
# Predict on the test set
new_y_pred = grid_search.predict(X_test)

# Calculate the mean squared error
new_mse = mean_squared_error(y_test, new_y_pred)
print(f'Mean Squared Error: {new_mse}')

# Calculate the R-squared value
new_r2 = r2_score(y_test, new_y_pred)
print(f'R-squared: {new_r2}')

Mean Squared Error: 2.8949836205207898e-05
R-squared: 0.9998630301824679


## Conclusions

- Several key features of the dataset, such as temperature, salinity, and oxygen levels, were identified and described, which are fundamental to understanding ocean conditions in the study area.
- Outliers were observed in some variables, which may indicate extreme oceanic events or measurement errors.
- Regression models were implemented to predict certain oceanic variables from others, using techniques such as linear regression and neural networks.
- The implementation of folders was essential for the development of an organized system in projects.